In [ ]:
import matplotlib.pyplot as plt

import igraph as ig
import networkx as nx
import pandas as pd
import numpy as np
import re
import swifter

from sklearn import preprocessing, model_selection
from sklearn.model_selection import cross_validate
import lightgbm as lgbm

from settings import file_names

# Datasets

In [ ]:
df_reviews = pd.read_csv(file_names['toronto_reviews_without_text'])
df_users = pd.read_csv(file_names['toronto_users'])
df_biz = pd.read_csv(file_names['toronto_businesses'])

In [ ]:
df_tips = pd.read_json(file_names['tip'], encoding='utf-8', lines=True)
df_tips = df_tips.loc[df_tips['user_id'].isin(df_users.user_id.unique()) & df_tips['business_id'].isin(df_biz.business_id.unique())]
print('Number of Tips: ', df_tips.shape[0])
df_tips.head(3)

In [ ]:
print('Number of reviews: ',df_reviews.shape[0])
print('Ratings describe: ', df_reviews['rating'].describe())
df_reviews.head(3)



In [ ]:
print('Number of users: ',df_users.shape[0])
df_users.head(3)

In [ ]:
print('Number of businesses: ',df_biz.shape[0])
df_biz.head(3)

### Small focus on coordinates --> We can use KMeans to keep track of the location

In [ ]:
coordinates = df_biz[['latitude', 'longitude']]
coordinates.plot.scatter('latitude','longitude')

In [ ]:
from sklearn.cluster import KMeans

clf = KMeans(n_clusters=10)
clf.fit(coordinates)
coordinates_clusters = clf.labels_
plt.scatter(x = coordinates['latitude'], y = coordinates['longitude'], c=coordinates_clusters)

# Collaborative Filtering - Rating Prediction

In [ ]:
from surprise import SVD, SVDpp, CoClustering, SlopeOne, KNNBasic, KNNWithZScore

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate as surprise_cv, GridSearchCV as surprise_gris_search_cv, train_test_split as surprise_test_train_split

df_surprise = df_reviews[['business_id', 'user_id', 'rating']]
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df_surprise, reader)
algo = SVD()

surprise_cv(algo, data, measures=['RMSE', 'MAE', 'fcp'], cv=5, verbose=3, n_jobs=3)

In [ ]:
trainset, testset = surprise_test_train_split(data, test_size=.25)
algo.fit(trainset)
predictions = algo.test(testset)
print(accuracy.fcp(predictions, verbose=True), accuracy.mae(predictions, verbose=True))

In [ ]:
predictions[:2]

# Baseline Binary Prediction - Collaborative Filtering

In [ ]:
from networkx.algorithms import bipartite

g = nx.Graph()
g.add_nodes_from(df_users.user_id.unique(), bipartite=0)
g.add_nodes_from(df_biz.business_id.unique(), bipartite=1)

g.add_edges_from([(user, business) for user, business
                           in zip(df_reviews.user_id, df_reviews.business_id)])

In [ ]:
binary_data = bipartite.biadjacency_matrix(g, df_users.user_id.unique()).todense()
df_binary = pd.DataFrame(binary_data)
df_binary['user_id'] = df_binary.index.values
df_binary.head(3)

In [ ]:
df_binary = pd.melt(df_binary, id_vars='user_id')
df_binary.rename({'variable':'business_id', 'value':'has_reviewed'}, axis='columns', inplace=True)

In [ ]:
df_binary

In [ ]:
def negative_sampling(df, ratio_zeros_on_ones):
    df_zeros = df_binary.loc[df_binary['has_reviewed'] == 0]
    df_ones = df_binary.loc[df_binary['has_reviewed'] != 0]
    df_zeros = df_zeros.sample(n=int(df_ones.shape[0]*ratio_zeros_on_ones))
    return pd.concat([df_zeros, df_ones]).sample(frac=1).reset_index(drop=True)

In [ ]:
reader = Reader(rating_scale=(0, 1))

In [ ]:
pd.read_csv('results_grid_search_1.csv')

# Binary Prediction - Using Graph Analysis
- Community detection
    - User --[Friends]-->User
    - User --[Reviewed]-->Restaurant
    - User --[Friends]-->User & User --[Reviewed]-->Restaurant
- Build feature 